In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

# Data Sampling 

In [2]:
def sample_data(m, n, winnow = False):
  #returns dataset sampled uniformly at random from {-1,1}^n of dimension n and size m
  if winnow:
    return np.random.choice([0,1], size = (m, n))
  else:
    return np.random.choice([-1,1], size = (m, n))

In [3]:
def gen_error(y_hat, y):
  #calculates generalisation error of prediction
  return  1 - (y_hat == y).sum()/y_hat.size

# Perceptron

In [4]:
def perceptron(X_train, X_test):
  
    w = np.ones(X_train.shape[1])
    sign = lambda x: -1 if x <= 0 else 1
    #loop through training data
    for train_index in range(X_train.shape[0]):
    
    #PREDICTION STEP
        y_hat = sign(w.T @ X_train[train_index, :])
    #UPDATE STEP 
        if X_train[train_index, 0] == y_hat:
            continue
        else:
            w += X_train[train_index, 0] * X_train[train_index, :]



    y_hat = np.array([w.T @ X_test[i,:] for i in range(X_test.shape[0])])
    y_hat = np.array(list(map(sign , y_hat)))
  
    return gen_error(y_hat, X_test[:, 0])

  


# Winnow 

In [5]:
def winnow(X_train, X_test):
    #train winnow classifier and calculate generalisation error on test set
    n = X_train.shape[1]
    w = np.ones(n)
    pred = lambda x: 0 if x < n else 1
  
    #loop through training data
    for train_index in range(X_train.shape[0]):
    
    #PREDICTION STEP

        y_hat = pred(w.T @ X_train[train_index, :])
    
        y = X_train[train_index, 0]
    
    #UPDATE STEP
        if y == y_hat:
            continue
        else: #if mistake, apply update rule for each element of w
            for i in range(len(w)):
                w[i] = w[i]*(2.0**((y - y_hat)*X_train[train_index, i]))

    y_hat = np.array([w.T @ X_test[i,:] for i in range(X_test.shape[0])])
    y_hat = np.array(list(map(pred , y_hat)))
  
    return gen_error(y_hat, X_test[:, 0])


# Least Squares

In [6]:
def least_squares(X_train, X_test):
  #trains a least squares classifier on X_train and predicts on X_test with sign(w^T X_test)
  #returns generalisation error of the model
    w = np.linalg.lstsq(X_train, X_train[:, 0], rcond = None)[0]
    y_hat = np.array([w.T @ X_test[i,:] for i in range(X_test.shape[0])])
    y_hat = np.array(list(map(lambda x: -1 if x < 0 else (1 if x > 0 else np.random.choice([-1,1])), y_hat)))

    return gen_error(y_hat, X_test[:, 0])



# 1-NN

In [7]:
def one_NN_helper(X_train, y_train, x_test):
    distances = np.linalg.norm(X_train - x_test, axis = 1)#finds euclidean distances between a point x_test and every training point

    nn_index = distances.argmin() #find index of nearest neighbour (min distance), in the event of a tie, the first of the nearest neighbours is chosen
    pred = y_train[nn_index] #find corresponding classes of nearest neighbour

    return pred

In [8]:
def one_NN(X_train, X_test):
  #Calculates generalisation error for 1-NN model given a training set and test set
    y_hat = np.fromiter((one_NN_helper(X_train, X_train[:,0], x_test) for x_test in X_test), float)  

    return gen_error(y_hat, X_test[:, 0])

# Estimating Sample Complexity 

In [9]:
models = [least_squares, perceptron, winnow, one_NN]
model_names = ['least_squares', 'perceptron', 'winnow', '1-NN']

In [10]:
def mean_gen_error(model, m, n, train_iter, test_size):
    #returns mean generalisation error for model by iteratively sampling training and test sets, training the model and calculating the generalisation error, returns the mean error from the runs
    er = []
    for _ in range(train_iter):
        if model == winnow:
            train = sample_data(m, n, winnow = True)
            test = sample_data(test_size, n, winnow = True)
        else:
            train = sample_data(m, n)
            test = sample_data(test_size, n)
        er.append(model(train, test))
    return np.mean(er)

In [11]:
def estimate_m_range(model, n = 100):
  #given that we intend on finding our sample complexities using binary search, we want to know, roughly, where to set our upper limit for the search. 
    for m in range(1, 1000, 100):
        er = mean_gen_error(model, m, n, 5, 1000)
        if er <= 0.1:
            return m
    return 5000

In [12]:
#estimate upper limits for binary search
for i in range(len(models)):
    print(f'Upper limit for m model {model_names[i]}:  {estimate_m_range(models[i])}')

Upper limit for m model least_squares:  101
Upper limit for m model perceptron:  201
Upper limit for m model winnow:  101
Upper limit for m model 1-NN:  5000


In [13]:
def binary_search(model,n, train_iter, test_size, high):
  #for a given n, perform a binary search over values of m, searching for sample complexity value
    low = 1
    gen_error = 1
    #first check for 1-NN that our upper limit for the binary search is indeed high enough
    if model == one_NN:
        while mean_gen_error(model, high, n, train_iter, test_size) > 0.1:
            print(f'need upper bound greater than {high}')
            high *= 2
        
    while high - low > 1:
        m = (high + low)//2
        #get estimated generalisation error for this m
        er = mean_gen_error(model, m, n, train_iter, test_size)
        
       
        if er <= 0.1:
            #if it is less than 0.1 and the m directly below it has error greater than 0.1, we are done
            if mean_gen_error(model, m - 1, n, train_iter, test_size) > 0.1:
                return m
            else:
                high = m
        else:
            low = m
    return high

In [14]:
def estimate_sample_complexity(model,train_iter = 100, test_size = 100, max_n = 100):
    #for a given algorithm, loops through each n and finds the sample complexity for that n through binary search
    #returns list of sample complexities
    ts = time.time()
    #takes a model as a parameter, trains
    sample_complexity = []
    high = estimate_m_range(model, n = max_n)
    
    for n in range(1, max_n +1):
        T = min(2**n,100)
        #print(n)
        res = binary_search(model,n, train_iter, test_size, high)
        sample_complexity.append(res)

    print(f'Time Elapsed: {time.time() - ts} seconds')
    return sample_complexity

In [ ]:
#calculate sample complexities for each algorithm
#do for n = 1,...,100 for all but 1-NN where we stop at N = 20 due to computation time
np.random.seed(0)
ls_sample_complexity = estimate_sample_complexity(least_squares, train_iter = 100, test_size = 100, max_n = 100)
perceptron_sample_complexity = estimate_sample_complexity(perceptron, train_iter = 100, test_size = 100, max_n = 100)
winnow_sample_complexity = estimate_sample_complexity(winnow, train_iter = 100, test_size = 100, max_n = 100)
one_nn_sample_complexity = estimate_sample_complexity(one_NN, train_iter = 100, test_size = 100, max_n = 20)

Time Elapsed: 93.49324774742126 seconds


In [ ]:
fig, ax = plt.subplots(figsize = (10,7))

ax.plot(np.arange(1,101), ls_sample_complexity)
ax.set_xlabel('n', fontsize= 15);
ax.set_ylabel('m', fontsize = 15);
ax.set_title('Least Squares Sample Complexity', fontsize = 20);

In [ ]:
fig, ax = plt.subplots(figsize = (10,7))

ax.plot(np.arange(1,101), perceptron_sample_complexity)
ax.set_xlabel('n', fontsize= 15);
ax.set_ylabel('m', fontsize = 15);
ax.set_title('Perceptron Sample Complexity', fontsize = 20);

In [ ]:
fig, ax = plt.subplots(figsize = (10,7))

ax.plot(np.arange(1,101), winnow_sample_complexity)
ax.set_xlabel('n', fontsize= 15);
ax.set_ylabel('m', fontsize = 15);
ax.set_title('Winnow Sample Complexity', fontsize = 20);

In [ ]:
fig, ax = plt.subplots(figsize = (10,7))

ax.plot(np.arange(1,21), one_nn_sample_complexity)
ax.set_xlabel('n', fontsize= 15);
ax.set_ylabel('m', fontsize = 15);
ax.set_title('1-NN Sample Complexity', fontsize = 20);
plt.xticks(np.arange(1,21,1));

# c)

In [ ]:
#feature maps for fitting least squares model to sample complexities to estimate how each grows with n

def feature_map(X, transform = None):
    #adds bias term and transforms features
    m = len(X)
    bias = np.ones((m,1))
    if transform:
        X = transform(X).reshape(m, 1)
    else:
        X = np.array(X).reshape(m, 1)
    return np.hstack([bias, X])

In [ ]:
def fit_OLS(X, y, transform = None):
    #fit OLS regression on transformed data
    Z = feature_map(X, transform)
    w = np.linalg.lstsq(Z, y, rcond = None)[0]
    y_hats = np.array(([w.T @ Z[i,:] for i in range(Z.shape[0])]))
    return w, y_hats

In [ ]:
#fit least squares OLS
# we find our best fitting model is linear so just bias term added
w_ls, pred_ls = fit_OLS(np.arange(1,101,1), np.array(ls_sample_complexity).reshape(100,1))
print(w_ls)
fig, ax = plt.subplots()

#print equation for line of best fit and plot bound lines
plt.plot(ls_sample_complexity, label = 'sample complexity')
plt.plot(1.1*pred_ls, label = 'upper bound')
plt.plot(0.9*pred_ls, label = 'lower bound')
ax.set_xlabel('n', fontsize = 10)
ax.set_ylabel('m', fontsize = 10)
ax.set_title('Bounds on Least Squares Sample Complexity');
ax.legend();

In [ ]:
#fit for perceptron, plot results
w_perceptron, pred_perceptron = fit_OLS(np.arange(1,101,1), np.array(perceptron_sample_complexity).reshape(100,1))
print(w_perceptron)
fig, ax = plt.subplots()
plt.plot(perceptron_sample_complexity, label = 'sample complexity')
plt.plot(1.1*pred_perceptron, label = 'upper bound')
plt.plot(0.9*pred_perceptron, label = 'lower bound')
ax.set_xlabel('n', fontsize = 10)
ax.set_ylabel('m', fontsize = 10)
ax.set_title('Bounds on Perceptron Sample Complexity')
plt.legend();

In [ ]:
#fit for winnow, find our best fit with log transform of data
w_winnow, pred_winnow = fit_OLS(np.arange(1,101,1), np.array(winnow_sample_complexity).reshape(100,1), np.log)
print(w_winnow)
fig, ax = plt.subplots()

plt.plot(winnow_sample_complexity, label = 'sample complexity')
plt.plot(1.15*pred_winnow, label = 'upper bound')
plt.plot(0.85*pred_winnow, label = 'lower bound')
ax.set_xlabel('n', fontsize = 10)
ax.set_ylabel('m', fontsize = 10)
ax.set_title('Bounds on Winnow Sample Complexity');
ax.legend();

In [ ]:
#for 1-nn we transform our data exponentially to give an equation m = a2^(bn)
n = np.arange(1,21,1)
m = len(n)
bias = np.ones((m,1))
two_n = np.power(2,n).reshape(m, 1)
Z = np.hstack([bias,  n.reshape(m,1)])
#Z = n.reshape(m,1)
w_nn = np.linalg.lstsq(Z, np.log2(one_nn_sample_complexity), rcond = None)[0]
pred_nn = 2**np.array(([w_nn.T @ Z[i,:] for i in range( Z.shape[0])]))

In [ ]:
print(w_nn)

fig, ax = plt.subplots()
plt.plot(one_nn_sample_complexity, label = 'sample complexity')
plt.plot(1.15*pred_nn, label = 'upper bound')
plt.plot(0.8*pred_nn, label = 'lower bound')

ax.set_xlabel('n', fontsize = 10)
ax.set_ylabel('m', fontsize = 10)
ax.legend();
plt.xticks(np.arange(20),np.arange(1,21,1));
ax.set_title('Bounds on 1-NN Sample Complexity');
